## Import Dependencies  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## Preparation
Loading data from **MNist** dataset, and define image embeddings \\

In this case, we have two separated datasets, the unpartitioned and partitioned. We perform traditional training on unpartitioned dataset and perform federated learning on partitioned dataset. The partitioned dataset would be splitted into 10 parts.

In [2]:
# Define transformations to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values to range [-1, 1]
])

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


## Define Model
Define the basic CNN model

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Non-partition

In [4]:
# Train function
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

# Test function
def test(model, testloader, verbose=True):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if verbose: print(f'Accuracy on test set: {100 * correct / total}%')
    return 100 * correct / total

In [5]:
# Train non-partitioned model
non_partitioned_model = CNN()
non_partitioned_optimizer = optim.Adam(non_partitioned_model.parameters(), lr=0.001)
non_partitioned_criterion = nn.CrossEntropyLoss()
print("Training non-partitioned model...")
train(non_partitioned_model, trainloader, non_partitioned_criterion, non_partitioned_optimizer)
print("Testing non-partitioned model...")
test(non_partitioned_model, testloader)

Training non-partitioned model...
Epoch 1, Loss: 0.13072585075913618
Epoch 2, Loss: 0.04525260726154472
Epoch 3, Loss: 0.02975676988341147
Epoch 4, Loss: 0.02270343013832074
Epoch 5, Loss: 0.017511909474524145
Testing non-partitioned model...
Accuracy on test set: 98.34%


98.34

### Partitioned

In [6]:
# Train partitioned model (simulate federated learning)
partition_size = len(trainset) // 5
data_partitions = [torch.utils.data.Subset(trainset, range(i * partition_size, (i + 1) * partition_size))
                   for i in range(5)]

partitioned_models = []
partitioned_optimizers = []
for _ in range(5):
    model = CNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    print("Training partitioned model...")
    train(model, torch.utils.data.DataLoader(data_partitions[_], batch_size=32, shuffle=True), criterion, optimizer, epochs=8)
    partitioned_models.append(model)
    partitioned_optimizers.append(optimizer)
    
    
# Computing Weight Averaging
partition_accuracy = [test(model, testloader, verbose=False) for model in partitioned_models]
total_accuracy = sum(partition_accuracy)
weights = [accuracy / total_accuracy for accuracy in partition_accuracy]

# Aggregate model updates
print("Aggregating model updates...")
aggregated_model = CNN()
for params_target in aggregated_model.parameters():
    params_target.data *= 0

for i in range(5):
    weight = weights[i]
    for params_source, params_target in zip(partitioned_models[i].parameters(), aggregated_model.parameters()):
        params_target.data += weight * params_source.data

# # Average aggregated model parameters
# for params_target in partitioned_models[0].parameters():
#     params_target.data /= 5

aggregated_model = partitioned_models[0]

Training partitioned model...
Epoch 1, Loss: 0.31955566228181126
Epoch 2, Loss: 0.08461292896171412
Epoch 3, Loss: 0.055219712830148635
Epoch 4, Loss: 0.03426092845434323
Epoch 5, Loss: 0.027789159183157608
Epoch 6, Loss: 0.020011472643362747
Epoch 7, Loss: 0.015462552505535616
Epoch 8, Loss: 0.011508275477885035
Training partitioned model...
Epoch 1, Loss: 0.3461557805587848
Epoch 2, Loss: 0.08790151850382487
Epoch 3, Loss: 0.05038244249361257
Epoch 4, Loss: 0.03793011387391016
Epoch 5, Loss: 0.030070044342738887
Epoch 6, Loss: 0.014550301084615058
Epoch 7, Loss: 0.016365758438871125
Epoch 8, Loss: 0.00964873318686538
Training partitioned model...
Epoch 1, Loss: 0.3532926323811213
Epoch 2, Loss: 0.08434604577968519
Epoch 3, Loss: 0.057701850733409325
Epoch 4, Loss: 0.036235368460183964
Epoch 5, Loss: 0.025447153719064468
Epoch 6, Loss: 0.018597482095181476
Epoch 7, Loss: 0.014087760137578394
Epoch 8, Loss: 0.014806691641109258
Training partitioned model...
Epoch 1, Loss: 0.33536243538

In [7]:
# Test federated model
print("Testing aggregated model...")
test(aggregated_model, testloader)

Testing aggregated model...
Accuracy on test set: 98.51%


98.51

In [21]:
print(partition_accuracy)

[98.06, 98.46, 98.57, 98.03, 98.52]
